In [1]:
# RESOURCES:
#https://www.youtube.com/watch?v=VlnJCV_cy8I

In [1]:
import os
import sys
import clr
import pandas as pd

clr.AddReference("System")  # Make sure to add a reference to the System namespace
from System import String, Array


C:\Users\austin.paxton\AppData\Local\Temp\ipykernel_43940\3183132935.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

ETABS_dll_path = R"C:\Program Files\Computers and Structures\ETABS 20\ETABSv1.dll"

from enum import Enum

clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

# set the following path to the installed ETABS program directory
clr.AddReference(ETABS_dll_path)
from ETABSv1 import *

In [3]:
# key variables
ETABS_model_folder_path = R"test_models\ETABS_gravity"
ETABS_model_file_path = R"test_models\ETABS_gravity\2023-08-29_Gravity Model.EDB"
ProgramPath = R"C:\Program Files\Computers and Structures\ETABS 20\ETABS.exe"

In [4]:
# Boiler plate code to open ETABs File WITHOUT specifiying ETABs executible

# # create API helper object
# helper = cHelper(Helper())
# # create instance of ETABs object from specified path
# myETABSObject = cOAPI(
#                     helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject")
#                 )

# # start ETABS application
# myETABSObject.ApplicationStart()

# # create SapModel object
# SapModel = cSapModel(myETABSObject.SapModel)

# # initialize model
# SapModel.InitializeNewModel()

# # open EtAbs model
# File = cFile(SapModel.File)
# ret = File.OpenFile(ETABS_model_file_path)


TypeError: object does not implement cOAPI

In [5]:
# Boiler plate code to open ETABs File by specifiying ETABs executible

# create API helper object
helper = cHelper(Helper())
# create instance of ETABs object from specified path
myETABSObject = cOAPI(helper.CreateObject(ProgramPath))

# start ETABS application
myETABSObject.ApplicationStart()

# create SapModel object
SapModel = cSapModel(myETABSObject.SapModel)

# initialize model
SapModel.InitializeNewModel()

# open EtAbs model
File = cFile(SapModel.File)
ret = File.OpenFile(ETABS_model_file_path)


In [6]:
# set units
kip_ft_F = 4
ret = SapModel.SetPresentUnits(eUnits(kip_ft_F))
# run model (this will create the analysis model)
Analyze = cAnalyze(SapModel.Analyze)  ###
ret = Analyze.RunAnalysis()

In [7]:
Results = cAnalysisResults(SapModel.Results)
Setup = cAnalysisResultsSetup(Results.Setup)

In [8]:
# count Linear Static Load Cases
LinearStatic=1
LoadCases = cLoadCases(SapModel.LoadCases)
ret = LoadCases.Count(eLoadCaseType(LinearStatic))
print(f"Load Case qty: {ret}")

Load Case qty: 9


In [9]:
# get linear static load case names
LC_names = []
num_names = 0
[ret, num_names, LC_names] = LoadCases.GetNameList(num_names, LC_names, eLoadCaseType(LinearStatic))

print(ret)
print("\n")
print(num_names)
print("\n")
print(type(LC_names))
print(LC_names[1])
for name in LC_names:
    print(name)

ETABS_LC = [name  for name in LC_names]

print(ETABS_LC)


0


9


<class 'System.String[]'>
Mech
Const
Mech
Dead
Live
Roof Live
S Dead
EQ-X_R=5
EQ-Y_R=5
~LLRF
['Const', 'Mech', 'Dead', 'Live', 'Roof Live', 'S Dead', 'EQ-X_R=5', 'EQ-Y_R=5', '~LLRF']


In [10]:
FrameObj = cFrameObj(SapModel.FrameObj)

NumberNames = 0
MyNames = []
PropName = []
StoryName = []
PointName1 = []
PointName2 = []
Point1X = []
Point1Y = []
Point1Z = []
Point2X = []
Point2Y = []
Point2Z = []
Angle = []
Offset1X = []
Offset2X = []
Offset1Y = []
Offset2Y = []
Offset1Z = []
Offset2Z = []
CardinalPoint = []

[
    ret,
    NumberNames,
    MyNames,
    PropName,
    StoryName,
    PointName1,
    PointName2,
    Point1X,
    Point1Y,
    Point1Z,
    Point2X,
    Point2Y,
    Point2Z,
    Angle,
    Offset1X,
    Offset2X,
    Offset1Y,
    Offset2Y,
    Offset1Z,
    Offset2Z,
    CardinalPoint
] = FrameObj.GetAllFrames(
    NumberNames,
    MyNames,
    PropName,
    StoryName,
    PointName1,
    PointName2,
    Point1X,
    Point1Y,
    Point1Z,
    Point2X,
    Point2Y,
    Point2Z,
    Angle,
    Offset1X,
    Offset2X,
    Offset1Y,
    Offset2Y,
    Offset1Z,
    Offset2Z,
    CardinalPoint
)

In [11]:
print(ret)

0


In [12]:
# explore frame data
def convert_system_array_to_list(sys_str):
    p_str = [item for item in sys_str]
    return p_str


#create pandas data frame
data = {
    "MyNames": convert_system_array_to_list(MyNames),
    "PropName": convert_system_array_to_list(PropName),
    "StoryName": convert_system_array_to_list(StoryName),
    "PointName1": convert_system_array_to_list(PointName1),
    "PointName2": convert_system_array_to_list(PointName2),
    "Point1X": convert_system_array_to_list(Point1X),
    "Point1Y": convert_system_array_to_list(Point1Y),
    "Point1Z": convert_system_array_to_list(Point1Z),
    "Point2X": convert_system_array_to_list(Point2X),
    "Point2Y": convert_system_array_to_list(Point2Y),
    "Point2Z": convert_system_array_to_list(Point2Z),
    "Angle": convert_system_array_to_list(Angle),
    "Offset1X": convert_system_array_to_list(Offset1X),
    "Offset2X": convert_system_array_to_list(Offset2X),
    "Offset1Y": convert_system_array_to_list(Offset1Y),
    "Offset2Y": convert_system_array_to_list(Offset2Y),
    "Offset1Z": convert_system_array_to_list(Offset1Z),
    "Offset2Z": convert_system_array_to_list(Offset2Z),
    "CardinalPoint": convert_system_array_to_list(CardinalPoint)
}

frames_df = pd.DataFrame(data)
frames_df.head()

,MyNames,PropName,StoryName,PointName1,PointName2,Point1X,Point1Y,Point1Z,Point2X,Point2Y,Point2Z,Angle,Offset1X,Offset2X,Offset1Y,Offset2Y,Offset1Z,Offset2Z,CardinalPoint
0,47,24Col8ksi,Level_2,587,58,13.67,275.0,0.0,13.67,275.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1,50,24Col8ksi,Level_3,58,563,13.67,275.0,10.5,13.67,275.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
2,55,24Col8ksi,Level_2,588,31,13.67,245.0,0.0,13.67,245.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
3,58,24Col8ksi,Level_3,31,561,13.67,245.0,10.5,13.67,245.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,59,24Col8ksi,Level_2,2116,72,13.67,221.0,0.0,13.67,221.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10


In [28]:
# filter for columns (z is allways reserved for vetical axis in  ETABs)
cols_df = frames_df[(frames_df["Point1X"] == frames_df["Point2X"] ) & (frames_df["Point1Y"] == frames_df["Point2Y"]) ]
display(cols_df.head())

# find lowest level columns 
# in program User will be prompted to chose the level they want to transfer
cols_L2_df = cols_df[cols_df["StoryName"] == "Level_2"]

bot_cols = cols_df["MyNames"].unique().tolist()

def convert_to_system_string(p_list):
    return Array[String](p_list)


# print(type(convert_to_system_string(bot_cols[0])))

,MyNames,PropName,StoryName,PointName1,PointName2,Point1X,Point1Y,Point1Z,Point2X,Point2Y,Point2Z,Angle,Offset1X,Offset2X,Offset1Y,Offset2Y,Offset1Z,Offset2Z,CardinalPoint
0,47,24Col8ksi,Level_2,587,58,13.67,275.0,0.0,13.67,275.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1,50,24Col8ksi,Level_3,58,563,13.67,275.0,10.5,13.67,275.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
2,55,24Col8ksi,Level_2,588,31,13.67,245.0,0.0,13.67,245.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
3,58,24Col8ksi,Level_3,31,561,13.67,245.0,10.5,13.67,245.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,59,24Col8ksi,Level_2,2116,72,13.67,221.0,0.0,13.67,221.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10


In [15]:
Results = cAnalysisResults(SapModel.Results)
Setup = cAnalysisResultsSetup(Results.Setup)

#select Load Cases for output
ret = Setup.DeselectAllCasesAndCombosForOutput()
ret = Setup.SetCaseSelectedForOutput('Dead')
print(ret)

0


In [16]:
#get Frame Forces for single Column
ObjectElm = 0

 #declare variables 
Name = str(1292)
NumberResults = 0
Obj = []
ObjSta = []
Elm = []
ElmSta = []
LoadCase = []
StepType = []
StepNum = []
P = []
V2 = []
V3 = []
T = []
M2 = []
M3 = []




[
    ret,
    NumberResults,
    Obj,
    ObjSta,
    Elm,
    ElmSta,
    LoadCase,
    StepType,
    StepNum,
    P,
    V2,
    V3,
    T,
    M2,
    M3
]=Results.FrameForce(
    Name,
    eItemTypeElm(ObjectElm),
    NumberResults,
    Obj,
    ObjSta,
    Elm,
    ElmSta,
    LoadCase,
    StepType,
    StepNum,
    P,
    V2,
    V3,
    T,
    M2,
    M3
)


print(f"analysis for: {Name}")
print(f"ret: {ret}")
    

analysis for: 1292
ret: 0


In [17]:
# explore results from Frame element
print(ret)
print(NumberResults)
print(P[0])
print(max(P))

print(LoadCase[2])

0
3
-28.891710959787133
-27.853024439979134
Dead


In [31]:
# get max axial force for each Object Element in a list

def find_max_axial(frame_objs: list) -> dict:
    ObjectElm = 0
    P_max = {}
    for frame in frame_objs:
        #declare variables 
        Name = str(frame)
        NumberResults = 0
        Obj = []
        ObjSta = []
        Elm = []
        ElmSta = []
        LoadCase = []
        StepType = []
        StepNum = []
        P = []
        V2 = []
        V3 = []
        T = []
        M2 = []
        M3 = []


        [
        ret,
        NumberResults,
        Obj,
        ObjSta,
        Elm,
        ElmSta,
        LoadCase,
        StepType,
        StepNum,
        P,
        V2,
        V3,
        T,
        M2,
        M3
        ]=Results.FrameForce(
        Name,
        eItemTypeElm(ObjectElm),
        NumberResults,
        Obj,
        ObjSta,
        Elm,
        ElmSta,
        LoadCase,
        StepType,
        StepNum,
        P,
        V2,
        V3,
        T,
        M2,
        M3
        )
        if ret == 0 and NumberResults != 0:
            P_max[frame] =  abs(min(P))
        else:
            print(f"Bad Response for frame: {frame}")
    return P_max


P_max = find_max_axial(bot_cols)
print(P_max)
print(type(P_max))
print(len(P_max))

{'47': 130.33290834967914, '50': 78.60489864177308, '55': 118.13143507268502, '58': 73.06973734246439, '59': 44.822920428148855, '85': 331.4827610481437, '86': 326.53475293541203, '87': 277.0399699972391, '88': 272.0919618845085, '89': 251.92490177740572, '90': 246.97689366467432, '91': 277.1365325136093, '92': 272.1885244008779, '93': 212.85127215895812, '94': 207.90326404622675, '95': 72.58094227675143, '96': 40.01763774457665, '97': 244.0328725860746, '98': 144.04874555287458, '99': 224.12415866940702, '100': 137.42422912829082, '101': 140.78878118546933, '102': 66.71287192033235, '103': 101.38470246572332, '104': 55.97457859444884, '105': 306.0444546066722, '106': 255.2443452022577, '107': 383.4729998522865, '108': 324.7591546057352, '109': 418.1698293746199, '110': 337.8373324546428, '111': 299.29000217899176, '112': 259.42348803667784, '113': 84.89716887473482, '114': 53.25652369956458, '115': 241.17577047885345, '116': 146.86346638351833, '117': 302.9312821440851, '118': 184.008

In [33]:
# add max forces to bot cols pandas dictionary
pd.options.mode.copy_on_write = True # ensures a copy is returned rather than a view
cols_L2_df["P_max"] = cols_L2_df["MyNames"].map(P_max)
display(cols_L2_df.head())

,MyNames,PropName,StoryName,PointName1,PointName2,Point1X,Point1Y,Point1Z,Point2X,Point2Y,Point2Z,Angle,Offset1X,Offset2X,Offset1Y,Offset2Y,Offset1Z,Offset2Z,CardinalPoint,P_max
0,47,24Col8ksi,Level_2,587,58,13.67,275.0,0.0,13.67,275.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,130.332908
2,55,24Col8ksi,Level_2,588,31,13.67,245.0,0.0,13.67,245.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,118.131435
4,59,24Col8ksi,Level_2,2116,72,13.67,221.0,0.0,13.67,221.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,44.822920
5,85,24Col8ksi,Level_2,590,591,13.67,116.5,0.0,13.67,116.5,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,331.482761
7,87,24Col8ksi,Level_2,594,595,13.67,90.5,0.0,13.67,90.5,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,277.039970


In [34]:
# # exit application
ret = myETABSObject.ApplicationExit(False)
SapModel = None
myETABSObject = None

In [20]:
################################## CANT PASS LIST INTO FRAME FORCE MMETHOD ###########################################
#get Frame Forces for multiple Columns
ObjectElm = 2

 #declare variables 
Name = convert_to_system_string([str(1292), str(43)])
NumberResults = 0
Obj = []
ObjSta = []
Elm = []
ElmSta = []
LoadCase = []
StepType = []
StepNum = []
P = []
V2 = []
V3 = []
T = []
M2 = []
M3 = []




[
    ret,
    NumberResults,
    Obj,
    ObjSta,
    Elm,
    ElmSta,
    LoadCase,
    StepType,
    StepNum,
    P,
    V2,
    V3,
    T,
    M2,
    M3
]=Results.FrameForce(
    Name,
    eItemTypeElm(ObjectElm),
    NumberResults,
    Obj,
    ObjSta,
    Elm,
    ElmSta,
    LoadCase,
    StepType,
    StepNum,
    P,
    V2,
    V3,
    T,
    M2,
    M3
)


print(f"analysis for: {Name}")
print(f"ret: {ret}")

TypeError: No method matches given arguments for cAnalysisResults.FrameForce: (<class 'System.String[]'>, <class 'ETABSv1.eItemTypeElm'>, <class 'int'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>)